In [12]:
!nvidia-smi


Wed Jan 28 09:51:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             27W /   70W |    1192MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q transformers accelerate bitsandbytes peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00


In [3]:
import torch
import time
import os
import gc
import psutil
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

torch.set_default_dtype(torch.float16)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


In [5]:
model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="sdpa",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"

model.eval()
print("Model loaded successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Model loaded successfully


In [6]:
prompt = "Explain why torch.compile is useful for ML systems engineering."
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=256,
).to(device)


In [7]:
torch_compile_options = {
    "epilogue_fusion": True,
    "max_autotune": True,
    "shape_padding": True,
    "trace.enabled": True,
    "triton.cudagraphs": False,
}

# Compile MLP
import transformers.models.llama.modeling_llama as llama_modeling

_original_mlp_forward = llama_modeling.LlamaMLP.forward

@torch.compile(dynamic=True, options=torch_compile_options)
def compiled_llama_mlp(self, x):
    return _original_mlp_forward(self, x)

llama_modeling.LlamaMLP.forward = compiled_llama_mlp

# Compile RMSNorm
_original_rmsnorm_forward = llama_modeling.LlamaRMSNorm.forward

@torch.compile(dynamic=True, options=torch_compile_options)
def compiled_rmsnorm(self, hidden_states):
    return _original_rmsnorm_forward(self, hidden_states)

llama_modeling.LlamaRMSNorm.forward = compiled_rmsnorm

print("MLP and RMSNorm patched with torch.compile")


MLP and RMSNorm patched with torch.compile


In [8]:
torch.cuda.empty_cache()
gc.collect()

# Warmup
with torch.no_grad():
    _ = model.generate(**inputs, max_new_tokens=20)

torch.cuda.synchronize()
start = time.time()

with torch.no_grad():
    for _ in range(5):
        _ = model.generate(**inputs, max_new_tokens=20)

torch.cuda.synchronize()
eager_time = time.time() - start

eager_vram = torch.cuda.max_memory_allocated() / 1024**2

print(f"Eager inference time : {eager_time:.2f} sec")
print(f"Eager VRAM usage    : {eager_vram:.2f} MB")


W0128 09:45:09.889000 1802 torch/_inductor/debug.py:507] [0/0] model__0_inference_0 debug trace: /content/torch_compile_debug/run_2026_01_28_09_45_07_145251-pid_1802/torchinductor/model__0_inference_0.0
W0128 09:45:22.719000 1802 torch/_inductor/utils.py:1558] [1/0] Not enough SMs to use max_autotune_gemm mode
W0128 09:45:23.769000 1802 torch/_inductor/debug.py:507] [1/0] model__1_inference_1 debug trace: /content/torch_compile_debug/run_2026_01_28_09_45_07_145251-pid_1802/torchinductor/model__1_inference_1.1
W0128 09:45:27.060000 1802 torch/_inductor/debug.py:507] [1/1] model__2_inference_2 debug trace: /content/torch_compile_debug/run_2026_01_28_09_45_07_145251-pid_1802/torchinductor/model__2_inference_2.2


Eager inference time : 5.03 sec
Eager VRAM usage    : 1177.56 MB


In [9]:
# =========================
# Cell 7: Compiled Inference Benchmark
# =========================

torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()

# Warmup (important for torch.compile)
with torch.no_grad():
    _ = model.generate(**inputs, max_new_tokens=20)

torch.cuda.synchronize()

start = time.time()

with torch.no_grad():
    for _ in range(5):
        _ = model.generate(**inputs, max_new_tokens=20)

torch.cuda.synchronize()
compiled_time = time.time() - start

compiled_vram = torch.cuda.max_memory_allocated() / 1024**2

print(f"Compiled inference time : {compiled_time:.2f} sec")
print(f"Compiled VRAM usage    : {compiled_vram:.2f} MB")


Compiled inference time : 5.72 sec
Compiled VRAM usage    : 1022.23 MB


In [11]:
# =========================
# Cell 8: Final Results + Interpretation
# =========================

print("\n===== TASK F – COMPILED INFERENCE BENCHMARK =====")
print(f"Eager inference time    : {eager_time:.2f} sec")
print(f"Compiled inference time : {compiled_time:.2f} sec")
print(f"Eager VRAM usage        : {eager_vram:.2f} MB")
print(f"Compiled VRAM usage     : {compiled_vram:.2f} MB")

speedup = eager_time / compiled_time if compiled_time > 0 else 0
print(f"Speedup factor          : {speedup:.2f}x")

print("\n--- Interpretation ---")
if speedup >= 1.0:
    print(
        "torch.compile improved inference latency after warm-up. "
        "This indicates compute-bound execution where kernel fusion is beneficial."
    )
else:
    print(
        "torch.compile reduced memory usage but did not improve latency on this setup. "
        "On Tesla T4 with small batch size, compilation overhead outweighs compute savings, "
        "which is expected behavior."
    )



===== TASK F – COMPILED INFERENCE BENCHMARK =====
Eager inference time    : 5.03 sec
Compiled inference time : 5.72 sec
Eager VRAM usage        : 1177.56 MB
Compiled VRAM usage     : 1022.23 MB
Speedup factor          : 0.88x

--- Interpretation ---
torch.compile reduced memory usage but did not improve latency on this setup. On Tesla T4 with small batch size, compilation overhead outweighs compute savings, which is expected behavior.
